#Assignment 2
##ENV717- Spring 2023
---

Instructor: [Dalia Patino-Echeverri ](mailto:dalia.patino@duke.edu)

Author(s): [Mauricio Hernandez](mailto:mmh54@duke.edu)

[Github repository](https://github.com/dpatinoecheverri/ENV717_S23)

## Objective: 

This assignment offers opportunities to explore the structure of the U.S. electricity industry through data available in eGRID.
---


In [ ]:
#import libraries
import pandas as pd # this imports the pandas library which is necessary for data frames and calls that library
 # can call all the functions of this library with "pd."
import numpy as np # same for numpy

In [ ]:

#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


##1a.	(15%)
Download the file eGRID2020.xls from the eGRID website and use the information in the PLNT20 tab to fill out the following table:

Breakout of the U.S. into two regions: a) those with organized whole-sale electricity markets and b) those serve predominately by vertically-integrated utilities.

|                 	| **Number of states <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)** 	| **Capacity in 2020 (MW)** 	| **Generation in 2020 (MWh)** 	|
|-----------------	|----------------------	|---------------------------	|------------------------------	|
| **RTO/ISO**     	|                      	|                           	|                              	|
| **Non-RTO/ISO** 	|                      	|                           	|                              	|

<font size= "2"><a name="cite_note-1"></a>[1.](#cite_ref-1) For each of the 52 U.S. “states” determine whether they are a “ISO/RTO” or not by looking at their nameplate generation capacity. If 51% or more is associated to an ISO/RTO, then designate this state as a “restructured” state. Otherwise, designate it as state with vertically integrated utilities. </font>

In [ ]:
###############################################################################################################
## 1. Read eGRID20 file directly form EPA website
## Read the PLNT20 sheet of the excel file into a pandas DataFrame.
## A DataFrame is a two-dimensional labeled data structure with columns of potentially different types
## Specify that the headers are on the second row of the excel file (i.e., row 1 in Python)
## 2. Confirm we read all columns by printing the first five rows
###############################################################################################################
df_plnt =pd.read_excel("https://www.epa.gov/system/files/documents/2022-01/egrid2020_data.xlsx", 
                       sheet_name="PLNT20", keep_default_na=False, header=1)
df_plnt.head(5) 

,SEQPLT20,YEAR,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,SECTOR,...,PLWIPR,PLSOPR,PLGTPR,PLOFPR,PLOPPR,PLTNPR,PLTRPR,PLTHPR,PLCYPR,PLCNPR
0,1,2020,AK,7-Mile Ridge Wind Project,60814,Alaska Power and Telephone Co,219,Alaska Power and Telephone Co,219,Electric Utility,...,,,,,,,,,,
1,2,2020,AK,Agrium Kenai Nitrogen Operations,54452,Homer Electric Assn Inc,19558,Agrium US Inc,179,Industrial CHP,...,,,,,,,,,,
2,3,2020,AK,Alakanuk,57053,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,,,,,,,,,,
3,4,2020,AK,Allison Creek Hydro,58982,"Copper Valley Elec Assn, Inc",4329,"Copper Valley Elec Assn, Inc",4329,Electric Utility,...,0,0,0,0,0,0,1,0,0,1
4,5,2020,AK,Ambler,60243,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,0,0,0,0,0,1,0,0,1,0


In [ ]:
#Name of columns
# 2. Select Columns that are needed for the analysis
# Plant name: PNAME
# Plant state abbreviation: PSTATABB
# Utility name: UTLSRVNM
# Plant-level sector: SECTOR
# Balancing Authority Name: BANAME
# Plant associated ISO/RTO Territory: ISORTO
# Plant capacity factor: CAPFAC
# Plant nameplate capacity: NAMEPCAP
# Plant annual net generation (MWh): PLNGENAN

# 3. Convert numeric columns to numeric, probably not necesary.
df_plnt.CAPFAC=pd.to_numeric(df_plnt.CAPFAC)
df_plnt.NAMEPCAP=pd.to_numeric(df_plnt.NAMEPCAP)
df_plnt.PLNGENAN=pd.to_numeric(df_plnt.PLNGENAN)

#Select only the attributes required, the copy function is recommended, the code 
# works without this function
df_plnt_summ = df_plnt.loc[:,['PNAME', 'PSTATABB', 'UTLSRVNM','SECTOR','BANAME', 
                              'ISORTO', 'CAPFAC','NAMEPCAP', 'PLNGENAN']].copy()

#Show first 5 plants
df_plnt_summ.head(n=5)

,PNAME,PSTATABB,UTLSRVNM,SECTOR,BANAME,ISORTO,CAPFAC,NAMEPCAP,PLNGENAN
0,7-Mile Ridge Wind Project,AK,Alaska Power and Telephone Co,Electric Utility,No balancing authority,,NaN,1.8,NaN
1,Agrium Kenai Nitrogen Operations,AK,Agrium US Inc,Industrial CHP,No balancing authority,,NaN,21.6,NaN
2,Alakanuk,AK,"Alaska Village Elec Coop, Inc",Electric Utility,No balancing authority,,0.00000,2.6,0.0
3,Allison Creek Hydro,AK,"Copper Valley Elec Assn, Inc",Electric Utility,No balancing authority,,0.36544,6.5,20808.0
4,Ambler,AK,"Alaska Village Elec Coop, Inc",Electric Utility,No balancing authority,,0.09224,1.5,1212.0


In [ ]:
# 4. Filter out all the plants that had zero or blank capacity factor in 2020 
# (i.e., Column AA in PLNT20 is zero or blank).
df_plnt_summ.dropna(axis='rows', subset=['CAPFAC'],inplace=True)
df_plnt_summ = df_plnt_summ[(df_plnt_summ.CAPFAC > 0)].copy()

In [ ]:
#5. Label the plants that belong to an ISO/RTO
df_plnt_summ['is_iso_rto'] = 'Yes'
df_plnt_summ.loc[df_plnt_summ.ISORTO == '', 'is_iso_rto'] = 'No'
#create pivot table by state, with Nameplate capacity and Total generation. 
# Values aggregated by plants that belong to ISO/RTO or not
pt_iso = pd.pivot_table(df_plnt_summ, index="PSTATABB", columns='is_iso_rto', 
                        values=['NAMEPCAP', 'PLNGENAN'], fill_value=0, aggfunc=['sum'])
#drop index level in columns 
pt_iso.columns = pt_iso.columns.droplevel(level =0)
pt_iso.head(n=5)

NAMEPCAP               PLNGENAN              
is_iso_rto       No      Yes            No           Yes
PSTATABB                                                
AK           3025.2      0.0  6.162056e+06  0.000000e+00
AL          35588.3      0.0  1.368420e+08  0.000000e+00
AR           2847.6  14595.3  9.468833e+06  4.517243e+07
AZ          31585.1      0.0  1.093189e+08  0.000000e+00
CA          18664.8  71669.3  3.253016e+07  1.607009e+08

In [ ]:

#DEtermine in which states the total nameplate capacity of their plants mostly
# are under the jurisdiction of an ISO/RTO
pt_iso['percent_iso'] = 100*pt_iso[('NAMEPCAP','Yes')]  / (pt_iso[('NAMEPCAP','Yes')] +pt_iso[('NAMEPCAP','No')] )
# Add and initialize two columns to the pivot table.
# The columns "Number of States" will be useful later to count how many have restructured 
pt_iso['Number of States'] = 1
#The column "restructured" is to indicate for each state if it is restructured or not. Initalize in no and change later based on % of ISO/RTO
pt_iso['restructured'] = 'N'
pt_iso.loc[pt_iso.percent_iso >= 51, 'restructured'] = 'Y'

# Add capacity and generation by state 
pt_iso['Capacity in 2020 (MW)'] = pt_iso[('NAMEPCAP','Yes')] +pt_iso[('NAMEPCAP','No')]
pt_iso['Generation in 2020 (MWh)'] = pt_iso[('PLNGENAN','Yes')] +pt_iso[('PLNGENAN','No')]

#Group data by the states that are mostly reformed and not reformed
pt_iso_summary = pt_iso.groupby(['restructured']).sum()
pt_iso_summary.columns = pt_iso_summary.columns.droplevel(level =1)
pt_iso_summary.rename(index={'Y': 'RTO/ISO', 'N':'Non-RTO/ISO'}, inplace=True)
pt_iso_summary.index.name = 'Region'
pt_iso_summary[['Number of States',	'Capacity in 2020 (MW)','Generation in 2020 (MWh)']]

,Number of States,Capacity in 2020 (MW),Generation in 2020 (MWh)
Organization,,,
Non-RTO/ISO,26,581620.0,1.773328e+09
RTO/ISO,26,749886.2,2.254063e+09


## 2 (31%) 
Download data on state’s average retail electricity prices for 2022 published by the Energy Information Administration EIA <a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2). Then explore the average prices in the RTO/ISO and non RTO/ISO regions. Fill out the following table:

|  	| **Number of States <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1)** 	| **Average retail electricity price in 2022 <br> across states in <br> the Eastern and Western Interconnects <a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) (cents/kWh)** 	| **Minimum average retail electricity <br> price in 2022 across states in <br> the Eastern and Western Interconnects (cents/kWh)** 	| Maximum average retail electricity price <br>in 2022 across states in <br>the Eastern and Western Interconnects (cents/kWh) 	|
|---	|:---:	|:---:	|:---:	|---	|
| **RTO/ISO** 	|  	|  	|  	|  	|
| **Non-RTO/ISO** 	|  	|  	|  	|  	|

<font size= "2"><a name="cite_note-2"></a>[2.](#cite_ref-2): “Rankings: Average Retail Price of Electricity to Residential Sector, September 2022
(cents/kWh)” Available at https://www.eia.gov/state/rankings/?sid=US#/series/31 </font>

<font size= "2"><a name="cite_note-3"></a>[3.](#cite_ref-3): Same as above, for each of the states served mainly by the Eastern and Western Interconnects (i.e., excluding Alaska, Hawaii, Puerto Rico and Texas) determine whether they are a “ISO/RTO” or not by looking at their nameplate generation capacity in eGRID2020. This average is an unweighted average where the prices for all the states in the group are considered regardless of the magnitude of consumption/generation.</font>

In [ ]:
#Read CSV file with electricity prices
path = "/content/gdrive/MyDrive/ENV717-S2023/A02/"
df_eia_prices = pd.read_csv(path + 'Average Retail Price of Electricity to Residential Sector-StateRankings.csv', 
                            index_col = 'State', usecols = [0,1,2])

#delete Rank column
df_eia_prices.drop(columns='Rank', inplace =True)

df_eia_prices.head(n=5)

,Rank,"Average Retail Price of Electricity to Residential Sector, cents/kWh"
State,,
HI,1,45.69
NH,2,32.32
CT,3,28.84
MA,4,27.77
CA,5,26.66


In [ ]:
pt_restructured_state = pt_iso[['restructured']].copy()

# modify column names of dataframe, reduce one level
pt_restructured_state.columns = pt_restructured_state.columns.droplevel(level =1)
#merge dataframe of states restructuring statos with dataframe of prices.
pt_iso_price = pt_restructured_state.join(df_eia_prices)


#Exclude Alaska, Hawaii, Puerto Rico and Texas 
pt_iso_price.drop(['AK', 'PR', 'HI', 'TX'], axis = 0, inplace =True)
pt_iso_price.head()

,reformed,"Average Retail Price of Electricity to Residential Sector, cents/kWh"
PSTATABB,,
AL,N,15.66
AR,Y,13.00
AZ,N,13.27
CA,Y,26.66
CO,N,15.16


In [ ]:
#obtain average, minimum and maximum prices by region (RTO and non-RTO)
df_iso_price_results = pd.pivot_table(pt_iso_price, index="restructured", fill_value=0, aggfunc=['mean','min', 'max'])

#separately, obtain the number of states that belong to RTO and no-RTO regions
#The number of states will be different than in the previous question because we dropped four
#states (AK,HI, PR, and TX)
df_iso_count = pt_iso_price.groupby(['restructured'])['restructured'].count()
#Now ...
df_iso_count = df_iso_count.to_frame()

#combine information of the count of states and the 
df_iso_price_results = df_iso_count.join(df_iso_price_results)
df_iso_price_results.rename(index={'Y': 'RTO/ISO', 'N':'Non-RTO/ISO'}, inplace=True)
df_iso_price_results.index.name = 'Region'
df_iso_price_results

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
  return merge(


,reformed,"(mean, Average Retail Price of Electricity to Residential Sector, cents/kWh)","(min, Average Retail Price of Electricity to Residential Sector, cents/kWh)","(max, Average Retail Price of Electricity to Residential Sector, cents/kWh)"
Region,,,,
Non-RTO/ISO,23,13.363043,10.51,15.66
RTO/ISO,25,18.627600,13.00,32.32


##3. 
This part of the assignment provides you with the opportunity to refresh your knowledge on microeconomics in the context of markets for regular commodities.

###3a. (24%, each subproblem 3%) 
Problem 2.2. from Kirschen and Strbac. 



###3b. (15%, a;6%, b; 9%) 
Problem 2.3. from Kirschen and Strbac. 



###3c. (15%) 
Problem 2.4. from Kirschen and Strbac.

In [ ]:
#access path where workbook is stored
path = "/content/gdrive/MyDrive/ENV717_S23_Solutions/A01/"

pt_iso.to_csv( path + '1b_iso_rto_by_state.csv')  


In [ ]:
%%shell
jupyter nbconvert --to html "/content/gdrive/MyDrive/ENV717_S23_Solutions/A01/ENV717_A1_Solution.ipynb"

[NbConvertApp] Converting notebook /content/gdrive/MyDrive/ENV717_S23_Solutions/A01/ENV717_A1_Solution.ipynb to html
[NbConvertApp] Writing 334720 bytes to /content/gdrive/MyDrive/ENV717_S23_Solutions/A01/ENV717_A1_Solution.html
